In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyOAuth

#Carga las variables de entorno desde el archivo .env
load_dotenv()

# Configura las credenciales de Spotify
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
SPOTIFY_REDIRECT_URI = os.getenv('SPOTIFY_REDIRECT_URI', 'http://localhost:8888/callback')


In [2]:
#Inicializar spotify con las credenciales
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET"),
    redirect_uri=os.getenv("SPOTIFY_REDIRECT_URI"),
    scope="user-library-read"
))

Función para obtener las canciones más populares de un artista.
    
    Parámetros:
        artist_name (str): Nombre del artista.
        country (str): Código del país para los resultados. Default: "US".
    
    Retorna:
        pd.DataFrame: DataFrame con las canciones más populares del artista.

In [9]:
def get_artist_top_tracks(artist_name, country="US"): #Es una función donde entran #el nombre del artista y el país
    results= sp.search(q=f'artist:{artist_name}', type='artist')# 
    if not results['artists']['items']:
        print(f"No se encontró el artista: {artist_name}")
        return pd.DataFrame()

    artist = results['artists']['items'][0]  # Obtiene el primer artista de los resultados
    artist_id = artist['id']  # Obtiene el ID del artista

    print(f"Artista encontrado: {artist['name']} (ID: {artist_id})")

    # Obtiene las pistas más populares del artista
    top_tracks = sp.artist_top_tracks(artist_id, country=country)

    print(top_tracks['tracks'][0]['name'])  # Imprime el nombre de la primera pista popular
    print(top_tracks['tracks'][0]['popularity'])  # Imprime el nombre de la primera pista popular
    

    print(f"Total de pistas encontradas: {len(top_tracks['tracks'])}")
    if not top_tracks['tracks']:
        print(f"No se encontraron pistas populares para el artista: {artist_name}")
        return pd.DataFrame()
    # Crea una lista de diccionarios con la información de las pistas
    tracks_data = []
    for track in top_tracks['tracks']:
        track_info = {
            'song_name': track['name'],
            'popularity': track['popularity'],
            'album': track['album']['name'],
            'release_date': track['album']['release_date'],
            'preview_url': track['preview_url']
        }
        tracks_data.append(track_info)
    
    # Crea un DataFrame de pandas con la información de las pistas
    return pd.DataFrame(tracks_data)

sp.search(...)
Este método permite hacer búsquedas dentro de Spotify. Puedes buscar por:

artist (artistas)

track (canciones)

album

playlist, etc.

q = "artist:Bad Bunny"

Esto le dice a Spotify que solo busque artistas cuyo nombre coincida con "Bad Bunny".

Si solo usaras "Bad Bunny" buscaría canciones, artistas, álbumes, etc., pero al decir "artist:Bad Bunny" filtras directamente por artistas.

type='artist'
Le estás diciendo a Spotify que solo quieres resultados de tipo artista, no canciones ni álbumes.

In [11]:
if __name__ == "__main__":
    artist_name = input("Ingresa el nombre del artista: ")
    country = input("Ingresa el código del país (default: US): ") or "US"
    
    # Obtener las canciones del artista
    df = get_artist_top_tracks(artist_name, country)

    if not df.empty:
        # Guardar los datos en un archivo CSV
        output_file = f"../data/{artist_name.replace(' ', '_')}_top_tracks.csv"
        df.to_csv(output_file, index=False)
        print(f"Datos guardados en: {output_file}")
    else:
        print("No se generaron datos para guardar.")

Artista encontrado: De La Rose (ID: 54seKvtsZauR1iauN0ptpo)
Q U E V A S H A C E R H O Y ?
87
Total de pistas encontradas: 10
Datos guardados en: ../data/de_la_rose_top_tracks.csv
